In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

#create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader  = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"shape of X [N,C,H,W]: {X.shape}")
    print(f"shape of y: {y.shape} {y.dtype}")
    break


shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64


In [4]:
device = torch.accelerator.current_accelerator().type \
if torch.accelerator.is_available() else "cpu"
print(device)

mps


In [5]:
#define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size =len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        #compute predictoin error
        pred = model(X)
        loss = loss_fn(pred, y)

        #backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch +1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) ==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
            

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch P{t+1}\n------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epoch P1
------------------
loss: 2.296116 [   64/60000]
loss: 2.292609 [ 6464/60000]
loss: 2.269511 [12864/60000]
loss: 2.264470 [19264/60000]
loss: 2.257841 [25664/60000]
loss: 2.218248 [32064/60000]
loss: 2.229378 [38464/60000]
loss: 2.187785 [44864/60000]
loss: 2.183603 [51264/60000]
loss: 2.155919 [57664/60000]
Test Error: 
 Accuracy: 40.8%, Avg loss: 2.147223 

Epoch P2
------------------
loss: 2.158573 [   64/60000]
loss: 2.148476 [ 6464/60000]
loss: 2.085598 [12864/60000]
loss: 2.097984 [19264/60000]
loss: 2.057758 [25664/60000]
loss: 1.996744 [32064/60000]
loss: 2.023980 [38464/60000]
loss: 1.936931 [44864/60000]
loss: 1.940043 [51264/60000]
loss: 1.864518 [57664/60000]
Test Error: 
 Accuracy: 60.7%, Avg loss: 1.862222 

Epoch P3
------------------
loss: 1.901545 [   64/60000]
loss: 1.869333 [ 6464/60000]
loss: 1.745160 [12864/60000]
loss: 1.784448 [19264/60000]
loss: 1.685895 [25664/60000]
loss: 1.641756 [32064/60000]
loss: 1.664842 [38464/60000]
loss: 1.559571 [44864/60000]


In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.path")

Saved PyTorch Model State to model.path


In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
